# checkList for Model
## network alg
+ network init?
## loss
+ loss input, target, output? nn.CrossEntropyLoss, (N, C), (N), (N)
## optimizer
+ what optimizer? Adam
## training_step
+ forward and loss?
+ log?
+ return loss(must included, maybe state)?
## validation_step
+ inference and loss?
+ choose metrics ?
+ log?
## test_step
+ any special?

In [2]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pytorch_lightning as pl

#d2l
import d2l.torch as d2l

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For Progress Bar
from tqdm import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

# pl model API

## Training
+ train_step
+ log: step log and epoch log
+ every train_epoch_end

In [3]:
class LitClassifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)

        # logs metrics for each training_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss  # must return loss(including dict form)

    def on_train_epoch_end(self):
        all_preds = torch.stack(self.training_step_outputs)
        # do something with all preds

## Validation

In [ ]:
class LitModel(pl.LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)  # You can add metrics to measure
        self.log('loss', loss, on_step=False, on_epoch=True, prog_bar=True)


def on_validation_epoch_end(self):
    all_preds = torch.stack(self.validation_step_outputs)
    # do something with all preds

**It is very useful to use `validate` to check benchmark(in only one epoch)**

In [ ]:
model = LitModel()
trainer = pl.Trainer()
trainer.validate(model)

## Test

In [ ]:
class LitModel(pl.LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)  # You can add metrics to measure
        self.log('loss', loss, on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # often reuse validation_step
        self.validation_step(batch, batch_idx)

In [ ]:
# call after training
trainer = pl.Trainer()
trainer.fit(model)

# automatically auto-loads the best weights from the previous run
trainer.test(dataloaders=test_dataloader)

# or call with pretrained model
model = MyLightningModule.load_from_checkpoint(PATH)
trainer = Trainer()
trainer.test(model, dataloaders=test_dataloader)

## predict
+ predict calls `forward` by default

In [ ]:
class LitMCdropoutModel(pl.LightningModule):
    def __init__(self, model, mc_iteration):
        super().__init__()
        self.model = model
        self.dropout = nn.Dropout()
        self.mc_iteration = mc_iteration

    # call forward by default
    def predict_step(self, batch, batch_idx):
        # enable Monte Carlo Dropout
        self.dropout.train()

        # take average of `self.mc_iteration` iterations
        pred = torch.vstack([self.dropout(self.model(x)).unsqueeze(0) for _ in range(self.mc_iteration)]).mean(dim=0)
        return pred

In [4]:
# call after training
trainer = pl.Trainer()
trainer.fit(model)

# automatically auto-loads the best weights from the previous run
predictions = trainer.predict(dataloaders=predict_dataloader)

# or call with pretrained model
model = MyLightningModule.load_from_checkpoint(PATH)
trainer = Trainer()
predictions = trainer.predict(model, dataloaders=test_dataloader)


NameError: name 'Trainer' is not defined

In [ ]:
# use model after training or load weights and drop into the production system
model = ClassificationTask.load_from_checkpoint("best_model.ckpt")
x = ...
model.eval()
with torch.no_grad():
    y_hat = model(x)

## save Hyperparameters
+ use `self.params`

In [ ]:
class LitMNIST(pl.LightningModule):
    def __init__(self, layer_1_dim=128, learning_rate=1e-2):
        super().__init__()
        # call this to save (layer_1_dim=128, learning_rate=1e-4) to the checkpoint
        self.save_hyperparameters()

        # equivalent
        self.save_hyperparameters("layer_1_dim", "learning_rate")

        # ignore the layer_1_dim
        self.save_hyperparameters(ignore=["layer_1_dim"])


        # Now possible to access layer_1_dim from hparams
        self.hparams.layer_1_dim

## load_from_checkpoint

In [ ]:
# the excluded parameters were `loss_fx` and `generator_network`
model = LitMNIST.load_from_checkpoint(PATH, loss_fx=torch.nn.SomeOtherLoss, generator_network=MyGenerator())